In [3]:
import numpy as np
from scipy.integrate import solve_ivp, trapezoid
from scipy.linalg import eig

In [4]:
# Problem 1
K = 1.0
L = 4.0
x = np.arange(-L, L + 0.1, 0.1)

def F(t, Y, eps):
    y1, y2 = Y
    return [y2, (K*t*t - eps)*y1]

def shoot_res(eps):
    sL = np.sqrt(K*L*L - eps)
    y0 = [1.0, sL]
    sol = solve_ivp(lambda xx, YY: F(xx, YY, eps), (x[0], x[-1]), y0, t_eval=x, method="BDF", atol=1e-8, rtol=1e-8)
    y1, y2 = sol.y[0], sol.y[1]
    R = y2[-1] + np.sqrt(K*L*L - eps)*y1[-1]
    return y1, R

def refine_root(a, b, tol=1e-4, itmax=60):
    fa = shoot_res(a)[1]
    fb = shoot_res(b)[1]
    for _ in range(itmax):
        c = (a*fb - b*fa)/(fb - fa)
        yc, fc = shoot_res(c)
        if abs(fc) < tol:
            return c, yc
        a, fa, b, fb = b, fb, c, fc
    return c, yc

eps_grid = np.linspace(0.5, 12.0, 200)
vals = []
for i in range(len(eps_grid)-1):
    _, f1 = shoot_res(eps_grid[i])
    _, f2 = shoot_res(eps_grid[i+1])
    if f1 == 0.0:
        vals.append((eps_grid[i], shoot_res(eps_grid[i])[0]))
    elif f1*f2 < 0 and len(vals) < 5:
        e, y = refine_root(eps_grid[i], eps_grid[i+1], tol=1e-4)
        vals.append((e, y))
    if len(vals) == 5:
        break

vals = sorted(vals, key=lambda z: z[0])
phis = []
epsilons = []
for e, y in vals:
    nrm = np.sqrt(trapezoid(np.abs(y)**2, x))
    y = (np.abs(y)/nrm).reshape(-1,1)
    phis.append(y)
    epsilons.append(e)

A1, A2, A3, A4, A5 = phis
A6 = np.array(epsilons, dtype=float).reshape(-1,1)

In [6]:
# Problem 2
K = 1.0
L = 4.0
dx = 0.1
x = np.arange(-L, L + dx, dx)
N = x.size
idx = np.arange(1, N - 1)
xi = x[idx]

A = np.zeros((N - 2, N - 2))
A[0, 0] = 2.0 / 3.0 + (dx**2) * K * xi[0] ** 2
A[-1, -1] = 2.0 / 3.0 + (dx**2) * K * xi[-1] ** 2
A[0, 1] = 1.0
A[-1, -2] = 1.0
for i in range(1, N - 3):
    A[i, i] = 2.0 + (dx**2) * K * xi[i] ** 2
    A[i, i - 1] = -1.0
    A[i, i + 1] = -1.0

w, V = eig(A)
eps = (w.real) / (dx**2)
order = np.argsort(eps)
eps = eps[order][:5]
V = V[:, order][:, :5]

phi_list = []
for j in range(5):
    v = V[:, j].real
    s = np.sqrt(K * L * L - eps[j])
    phi0 = (4.0 * v[0] - v[1]) / (3.0 + 2.0 * dx * s)
    phiN = (4.0 * v[-1] - v[-2]) / (3.0 - 2.0 * dx * s)
    phi_full = np.concatenate(([phi0], v, [phiN]))
    nrm = np.sqrt(trapezoid(np.abs(phi_full) ** 2, x))
    phi_full = (np.abs(phi_full) / nrm).reshape(-1, 1)
    phi_list.append(phi_full)

A7, A8, A9, A10, A11 = phi_list
A12 = eps.reshape(-1, 1)


In [8]:
# Problem 3
K = 1.0
L = 2.0
dx = 0.1
x = np.arange(-L, L + dx, dx)
tol = 1e-4

def rhs(xx, Y, eps, gamma):
    y, yp = Y
    return [yp, (gamma*np.abs(y)**2 + K*xx*xx - eps)*y]

def integrate_on_grid(eps, gamma, A0):
    A = A0
    for _ in range(60):
        y0 = [A, A*np.sqrt(K*L*L - eps)]
        sol = solve_ivp(lambda xx, YY: rhs(xx, YY, eps, gamma),
                        (x[0], x[-1]), y0, method="BDF",
                        atol=1e-9, rtol=1e-9, max_step=dx)
        if sol.status < 0 or sol.t[-1] < x[-1]:
            sol = solve_ivp(lambda xx, YY: rhs(xx, YY, eps, gamma),
                            (x[0], x[-1]), y0, method="Radau",
                            atol=1e-9, rtol=1e-9, max_step=dx)
        y_raw, yp_raw, t_raw = sol.y[0], sol.y[1], sol.t
        y = np.interp(x, t_raw, y_raw)
        yp = np.interp(x, t_raw, yp_raw)
        nrm = np.sqrt(trapezoid(np.abs(y)**2, x))
        if abs(nrm - 1.0) < tol:
            return y, yp, A
        A = A / nrm
    return y, yp, A

def boundary_residual(yL, ypL, eps):
    s = np.sqrt(K*L*L - eps)
    return ypL + s*yL

def solve_mode(gamma, mode):
    e0 = 2.0*mode + 1.0
    e1 = e0 + 0.2
    y0, yp0, _ = integrate_on_grid(e0, gamma, 1.0)
    R0 = boundary_residual(y0[-1], yp0[-1], e0)
    y1, yp1, _ = integrate_on_grid(e1, gamma, 1.0)
    R1 = boundary_residual(y1[-1], yp1[-1], e1)
    for _ in range(60):
        e2 = (e0*R1 - e1*R0)/(R1 - R0)
        y2, yp2, _ = integrate_on_grid(e2, gamma, 1.0)
        R2 = boundary_residual(y2[-1], yp2[-1], e2)
        if abs(R2) < tol:
            y2 = (np.abs(y2)/np.sqrt(trapezoid(np.abs(y2)**2, x))).reshape(-1,1)
            return e2, y2
        e0, R0 = e1, R1
        e1, R1 = e2, R2
    y2 = (np.abs(y2)/np.sqrt(trapezoid(np.abs(y2)**2, x))).reshape(-1,1)
    return e2, y2

e_p, phi1_p = solve_mode(0.05, 0)
e_p2, phi2_p = solve_mode(0.05, 1)
A13, A14 = phi1_p, phi2_p
A15 = np.array([e_p, e_p2], dtype=float).reshape(-1,1)

e_n, phi1_n = solve_mode(-0.05, 0)
e_n2, phi2_n = solve_mode(-0.05, 1)
A16, A17 = phi1_n, phi2_n
A18 = np.array([e_n, e_n2], dtype=float).reshape(-1,1)
